In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# load model and tokenizer
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
falcon_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    device=0  # use GPU if available, set to -1 for CPU
)

/soft/applications/miniconda3/conda_pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]


# Zero-shot Example
* Model has the ability to handle new tasks without prior examples.
* Evaluates model's interpretation level, generates answer based only on task details.

In [2]:
zero_shot_prompt = "Add all the odd numbers in the list: 4, 8, 9, 15, 12, 2, 1. What is the total?"

response_zero_shot = falcon_pipeline(
    zero_shot_prompt,
    max_new_tokens=50,  # define the generated output's length
    temperature=0.5,
    truncation=True,
    do_sample=True,  # enable sampling
    pad_token_id=tokenizer.eos_token_id
)
print(response_zero_shot[0]['generated_text'])

Add all the odd numbers in the list: 4, 8, 9, 15, 12, 2, 1. What is the total?
The sum of all odd numbers in the list is 20 (4 + 8 + 9 + 15 + 12 + 2 + 1).


# Few-shot Example
* Shows the model's generalization level when it's provided examples.

In [3]:
few_shot_prompt = """Here are examples of how to compute the sum of odd numbers:
Example 1: The odd numbers in [3, 4, 7, 8] are [3, 7]. The sum is 10.
Example 2: The odd numbers in [2, 5, 11, 14] are [5, 11]. The sum is 16.
Question: What is the sum of the odd numbers in [4, 8, 9, 15, 12, 2, 1]?"""

response_few_shot = falcon_pipeline(
    few_shot_prompt,
    max_new_tokens=100,  # generated output's length
    temperature=0.5,
    truncation=True,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
print(response_few_shot[0]['generated_text'])

Here are examples of how to compute the sum of odd numbers:
Example 1: The odd numbers in [3, 4, 7, 8] are [3, 7]. The sum is 10.
Example 2: The odd numbers in [2, 5, 11, 14] are [5, 11]. The sum is 16.
Question: What is the sum of the odd numbers in [4, 8, 9, 15, 12, 2, 1]?
Answer: The sum of the odd numbers in [4, 8, 9, 15, 12, 2, 1] is 17.

Explanation:
1. The odd numbers in [4, 8, 9, 15, 12, 2, 1] are [4, 9, 12, 15, 2, 1].
2. The sum of these odd numbers is


# Chain-of-thought (CoT) Example
* Model's step-by-step reasoning capabilites are evaluated.
* Better debugging of logic failure for incorrect answers.

In [4]:
cot_prompt = """Follow these steps to compute the sum of odd numbers:
Step 1: Identify and list the odd numbers in the list.
Step 2: Add the identified odd numbers."""

response_cot = falcon_pipeline(
    cot_prompt,
    max_new_tokens=100,
    temperature=0.5,
    truncation=True,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
print(response_cot[0]['generated_text'])

Follow these steps to compute the sum of odd numbers:
Step 1: Identify and list the odd numbers in the list.
Step 2: Add the identified odd numbers.
Step 3: Repeat steps 1 and 2 until all the odd numbers have been added.
Step 4: The sum of odd numbers is the sum of the last two odd numbers in the list.


# Few-shot CoT Example
* The model combines few-shot examples advantages and step-by-step reasoning.

In [5]:
few_shot_cot_prompt = """Here are examples of how to compute the sum of odd numbers step-by-step:
Example 1:
List: [3, 4, 7, 8]
Step 1: Identify the odd numbers: [3, 7].
Step 2: Compute the sum: 10.

Example 2:
List: [2, 5, 11, 14]
Step 1: Identify the odd numbers: [5, 11].
Step 2: Compute the sum: 16.

Question:
List: [4, 8, 9, 15, 12, 2, 1]
Step 1:
"""
response_few_shot_cot = falcon_pipeline(
    few_shot_cot_prompt,
    max_new_tokens=100,  # Reduce token limit
    temperature=0.5,
    truncation=True,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
print(response_few_shot_cot[0]['generated_text'])

Here are examples of how to compute the sum of odd numbers step-by-step:
Example 1:
List: [3, 4, 7, 8]
Step 1: Identify the odd numbers: [3, 7].
Step 2: Compute the sum: 10.

Example 2:
List: [2, 5, 11, 14]
Step 1: Identify the odd numbers: [5, 11].
Step 2: Compute the sum: 16.

Question:
List: [4, 8, 9, 15, 12, 2, 1]
Step 1:
List: [4, 8, 9, 15, 12, 2, 1]
Step 2:
Step 3:

The sum of odd numbers is 15.


# Low Temperature Example
* Produces safer outputs that are useful for structured tasks.

In [6]:
low_temp_prompt = "Write a poem about a land of candy."
response_low_temp = falcon_pipeline(
    low_temp_prompt,
    max_new_tokens=150,
    temperature=0.2,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
print(response_low_temp[0]['generated_text'])

Write a poem about a land of candy.
A land of sweet delights,
Where every taste is found.
Sugar-coated confections,
In every shape and form.

Lollipops of every hue,
Dipped in chocolate, pure and true.
Gummy bears that tingle,
With every bite, they tingle too.

Mouth-watering hard candies,
In every flavor, from sour to candy.
Fruity chews that make you smile,
With every chew, your smile will multiply.

A land of candy, so delightful,
Where every sweet tooth can find its delight.


# High Temperature Example
* Produces more creative ouputs that are useful for open-ended tasks.

In [7]:
high_temp_prompt = "Write a poem about a land of candy."
response_high_temp = falcon_pipeline(
    high_temp_prompt,
    max_new_tokens=150,
    temperature=0.8,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
print(response_high_temp[0]['generated_text'])

Write a poem about a land of candy.
Once upon a sweet kingdom did reside,
Where every day was filled with sugary glee.
Candy trees grew tall and candy-coated hills,
Dusting each sweet surface with a magical thrill.

In this confectionery wonderland,
One could find a treat to make them feel grand.
Gummy bears, sour patches, and lollipops galore,
Mouth-watering delicacies, a sugary shore.

Candy castles, spun sugar, and rainbow-filled skies,
The land of sweet is where each fantasy thrives.
